In [ ]:
%store -r

In [ ]:
finetuned_model_uri

### Test LMI DLC based SageMaker endpoint in local environment

- Before deploy endpoint, we can test it local env.
- Refer to the `local_lmi_endpoint.sh` file

In [ ]:
import sagemaker
from sagemaker.utils import name_from_base
from sagemaker import image_uris

In [ ]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
sm_client = sagemaker_session.sagemaker_client
sm_runtime_client = sagemaker_session.sagemaker_runtime_client
default_bucket = sagemaker_session.default_bucket()

In [ ]:
llm_engine = "deepspeed"
# llm_engine = "fastertransformer"

In [ ]:
framework_name = f"djl-{llm_engine}"
inference_image_uri = image_uris.retrieve(
    framework=framework_name, region=sagemaker_session.boto_session.region_name, version="0.25.0"
)
print(f"Inference container uri: {inference_image_uri}")

In [ ]:
src_dir_name = f"ko-llm-med-src"
s3_target = f"s3://{default_bucket}/llm/med-finetune/code/"

In [ ]:
!rm -rf {src_dir_name}.tar.gz
!tar zcvf {src_dir_name}.tar.gz {src_dir_name} --exclude ".ipynb_checkpoints" --exclude "__pycache__"
!aws s3 cp {src_dir_name}.tar.gz {s3_target}

In [ ]:
model_uri = f"{s3_target}{src_dir_name}.tar.gz"
print(model_uri)

In [ ]:
model_name = name_from_base(f"ko-med-lmidlc")
print(model_name)

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": inference_image_uri, "ModelDataUrl": model_uri},
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

In [ ]:
instance_type = "ml.g4dn.xlarge"

endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": instance_type,
            "InitialInstanceCount": 1,
            "ContainerStartupHealthCheckTimeoutInSeconds": 450,
        },
    ],
)
print(endpoint_config_response)

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

In [ ]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

In [ ]:
import json

In [ ]:
# prompt = "초록색 가래가 계속 나오고 기침이 나오는데, 큰 병은 아닐까요?"
# prompt = "항문압 측정 검사에서 항문 압력이 증가하는 경우는 어떤 경우일까요?"
# prompt = "55세 남자가 위암으로 위절제술을 받았다. 수술 후 3일째 혈색소가 6.7 g/dL,로 감소하여 농축적혈구 수혈을 시작하였다. 수혈 도중 갑자기 오한과 발열이 발생하였으며, 복부 피부절개 부위에서 혈성 삼출물이 보이고 수혈 주사부위에 통증이 생겼다. 혈압 100/60 mmHg, 맥박 102회/분, 호흡 24회/분, 체온 38.2도 이다. 처치는?"
# prompt = "감염병 유행 역학조사를 수행하면서 인구학적 특성별, 발생 시기별로 환자발생 점지도(spot map) 를 작성하였다. 이를 통해 알 수 있는 것은?"
# prompt = "42세 남자가 1개월 전부터 숨이 많이 찬다며 병원에 왔다. 4년 전부터 숨이 찼다고 한다. 1주 전부터는 기침도 하고, 밤에 잘 때 쌕쌕거리는 소리도 난다고 한다. 20갑 ∙ 년의 흡연자이다. 혈압 120/70 mmHg, 맥박 88회/분, 호흡 20회/분, 체온 36.4℃이다. 가슴 청진에서 호흡음은 정상이다. 가슴 X선사진은 정상이다. 폐기능검사 결과는 다음과 같다. 다음 검사는?강제폐활량: 3.0 L (예측치의 92%) 1초간 강제날숨량: 2.7 L (예측치의 90%)"
# prompt = "1개월 남아가 12시간 전부터 피부색이 창백하고 얼룩덜룩하게 변하여 병원에 왔다. 하루 전부터 많이 보채면서 한 번에 분유를 30 mL도 못 먹었지만 배가 불러보였다고 한다. 어딘지 평소와 다르게 보였고, 집에서 측정한 체온은 35.9℃부터 37.7℃까지 오르내렸다. 임신나이 39주, 출생체중 3,300 g, 질분만으로 태어나서, 아프기 전에는 잘 지내왔었다. 혈압 65/45 mmHg, 맥박 155회/분, 호흡 60회/분, 체온 39.2℃이다. 피부색은 창백하고, 진찰하는 동안에도 많이 처져서 잘 울지 않는다. 검사는?" 
# prompt = "우리나라는 환자의 수도권 집중 방지와 지방의 진료 역량 강화를 위하여 수도권을 제외한 시도에 권역 단위로 전문질환센터를 지정하여 지원하고 있다. 권역 전문질환센터의 설립 취지를 반영하는 병원 관리 평가지표는?"
# prompt = "65세 이상 노인에게서 인플루엔자 예방접종이 폐렴으로 인한 입원율을 낮추는 효과가 있는지 확인하려고 한다. 일정기간 폐렴으로 인한 입원 환자 수는 접종군 4,000명 중 20명, 비접종군 2,000명 중 40명이다. 인플루엔자 예방접종의 폐렴 입원에 대한 예방효과는?"
# prompt = "펜션에서 투숙객 3명이 혼수상태로 발견되었다. 특별한 외상은 관찰되지 않았으며, 혈액검사에서 일산화탄소헤모글로빈(COHb) 은 40% 이상이었다. 치료는?"
# prompt = "미생물연구실에서 실험을 하던 25세 여자가 눈의 충혈, 이물감, 시림, 통증으로 병원에 왔다. 이 여자는 보안경을 착용하지 않은 채 무균실험대 위에서 살균등을 켜고 실험을 하였다. 노출이 의심되는 방사선은?"
# prompt = "42세 남자가 화를 못 참겠다며 병원에 왔다. 1개월 전에 골육종으로 오른쪽 다리를 절단해야 한다는 진단을 받았다. 상급종합병원에서 다시 한 번 검사를 받았으나 결과는 같았다. 왜 이런 일이 생긴 건지 믿을 수가 없다며 화를 내다가도 아직 7개월밖에 안 된 아들을 생각하면 갑자기 눈물이 나와 멈출 수가 없었다. 직장에서는 사소한 일에도 화가 치밀어 동료들과 다툼이 잦아졌고 업무에도 실수가 많아져 사람이 갑자기 변했다는 소리를 들었다고 한다. 밤에는 미래에 대한 걱정으로 잠을 거의 자지 못했고 식욕이 없어 하루에 한 끼를 겨우 먹었다. 치료는?"
prompt = "나이가 만 38세 정도 되는데 운동하다가 허리를 삐끗한 것 같습니다. 통증이 계속해서 줄어들지 않는데 어떻게 할까요?"

# prompt = "python과 javascript와 java는 어떤 점이 다른가요?"
# prompt = "전주에서 여행할 만한 곳 좀 추천해주세요."

In [ ]:
# instruction = "아래 질문에 대해서 최대한 길고 자세하게 답변해 주세요."

instruction = """
당신은 똑똑한 의사입니다. 질문에 대해서 최대한 자세하게 어떤 병인지, 치료법은 무엇인지 환자에게 설명하듯 친절하게 설명해 주세요.
"""



In [ ]:
payload = {
            "input_text": prompt,
            "instruction": instruction,
            "parameters": {
                "max_new_tokens": 4096,
                "temperature": 0.9,
                "top_p": 0.9,
            },
        }

In [ ]:
%%time

response_model = sm_runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(payload),
    ContentType="application/json",
)

In [ ]:
output = str(response_model["Body"].read(), "utf-8")
print(output)

## 참고

- Local LMI DLC 배포 후 테스트하는 경우

In [ ]:
import requests

endpoint = "http://127.0.0.1:8080/invocations"

# data = json.dumps(payload)
data = payload
headers = {"content-type": "application/json"}

response_model = requests.post(
    endpoint, headers=headers, json=data)

In [ ]:
print(response_model.text)